In [1]:
from flashtext import KeywordProcessor
import spacy
from spacy import displacy
from spacy.lang.zh import Chinese
from spacy.tokens import Doc, Span

In [2]:
nlp = Chinese()

In [3]:
import os
import sys

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
w_dir

'/work/fgc_support_retri'

In [7]:
sys.path.append(w_dir+'/fgc_support_retri')
import config
from utils import *

In [62]:
fgc_data = json_load(config.FGC_DEV)

In [63]:
def match_keyword(question, keywords):
    keyword_processor = KeywordProcessor()
    for keyword in keywords:
        keyword_processor.add_keyword(keyword)
    return keyword_processor.extract_keywords(question, span_info=True)

In [64]:
fgc_data[0]['SENTS'][1:7]

[{'text': '眉州眉山（今四川省眉山市）人，', 'start': 25, 'end': 40},
 {'text': '北宋时著名的文学家、政治家、艺术家、医学家。', 'start': 40, 'end': 62},
 {'text': '字子瞻，一字和仲，', 'start': 62, 'end': 71},
 {'text': '号东坡居士、铁冠道人。', 'start': 71, 'end': 82},
 {'text': '嘉佑二年进士，', 'start': 82, 'end': 89},
 {'text': '累官至端明殿学士兼翰林学士，', 'start': 89, 'end': 103}]

In [65]:
fgc_data[0]['QUESTIONS'][0]

{'QID': 'D001Q01',
 'QTYPE': '基礎題',
 'ATYPE': 'Date-Duration',
 'AMODE': ['Single-Span-Extraction'],
 'QTEXT': '蘇東坡在中國歷史上，是哪一個朝代的人？',
 'QTEXT_CN': '苏东坡在中国历史上，是哪一个朝代的人？',
 'SENTS': [{'text': '苏东坡在中国历史上，', 'start': 0, 'end': 10},
  {'text': '是哪一个朝代的人？', 'start': 10, 'end': 19}],
 'SHINT': [0, 1, 2, 4],
 'ANSWER': [{'ATEXT': '北宋',
   'ATOKEN': [{'text': '北宋', 'start': 40, 'end': 42}]}],
 'ASPAN': [{'text': '苏轼', 'start': 0, 'end': 2},
  {'text': '东坡居士', 'start': 72, 'end': 76},
  {'text': '北宋', 'start': 40, 'end': 42}]}

In [66]:
import jieba
import jieba.posseg as pseg
words = pseg.cut("苏东坡在宋哲宗时期,曾出任过哪些官职?", use_paddle=True)
for word, flag in words:
    print('%s %s' % (word, flag))

苏东坡 ns
在 p
宋哲宗 nr
时期 n
, x
曾 d
出任 v
过 ug
哪些 r
官职 n
? x


In [67]:
for d in fgc_data:
    for q in d['QUESTIONS']:
        q_text = q['QTEXT_CN'] 
        
        try:
            keywords = [aspan['text'] for aspan in q['ASPAN']]
        except:
            print(q)
            continue
        
        kspan = []
        entities = []
        doc = Doc(nlp.vocab, words=list(q_text))
        for kmatch in match_keyword(q_text, keywords):
            kspan.append({'text': kmatch[0], 'start': kmatch[1], 'end': kmatch[2]})
            entities.append(Span(doc, kmatch[1], kmatch[2], "keyword"))
        doc.ents = entities
        q['keywords'] = kspan
#         displacy.render(doc, jupyter=True, style='ent')

{'QID': 'D001Q11', 'QTYPE': '申論', 'ATYPE': 'Event', 'AMODE': ['Single-Span-Extraction'], 'QTEXT': '蘇東坡為何被後人認為是文學藝術史上的通才?', 'QTEXT_CN': '苏东坡为何被后人认为是文学艺术史上的通才?', 'SENTS': [{'text': '苏东坡为何被后人认为是文学艺术史上的通才?', 'start': 0, 'end': 21}], 'SHINT': []}
{'QID': 'D049Q04', 'QTYPE': '申論', 'ATYPE': 'Event', 'AMODE': ['Single-Span-Extraction'], 'QTEXT': '「雅婷逐字稿」的命名起源為何?', 'QTEXT_CN': '「雅婷逐字稿」的命名起源为何?', 'SENTS': [{'text': '「雅婷逐字稿」的命名起源为何?', 'start': 0, 'end': 15}], 'SHINT': []}
{'QID': 'D073Q12', 'QTYPE': '申論', 'ATYPE': 'Object', 'AMODE': ['Single-Span-Extraction'], 'QTEXT': '台灣對於鯨鯊保育做了哪些努力?', 'QTEXT_CN': '台湾对于鲸鲨保育做了哪些努力?', 'SENTS': [{'text': '台湾对于鲸鲨保育做了哪些努力?', 'start': 0, 'end': 15}], 'SHINT': []}
{'QID': 'D103Q07', 'QTYPE': '申論', 'ATYPE': 'Kinship', 'AMODE': ['Kinship'], 'QTEXT': '什麼是「325停課標準」?', 'QTEXT_CN': '什么是「325停课标准」?', 'SENTS': [{'text': '什么是「325停课标准」?', 'start': 0, 'end': 13}], 'SHINT': []}
{'QID': 'D289Q07', 'QTYPE': '申論', 'ATYPE': 'Kinship', 'AMODE': ['Kinship'], 'QTEXT': '人類藉由普羅米修斯認識了火，發現它不

In [68]:
fgc_data[0]

{'DID': 'D001',
 'QUESTIONS': [{'QID': 'D001Q01',
   'QTYPE': '基礎題',
   'ATYPE': 'Date-Duration',
   'AMODE': ['Single-Span-Extraction'],
   'QTEXT': '蘇東坡在中國歷史上，是哪一個朝代的人？',
   'QTEXT_CN': '苏东坡在中国历史上，是哪一个朝代的人？',
   'SENTS': [{'text': '苏东坡在中国历史上，', 'start': 0, 'end': 10},
    {'text': '是哪一个朝代的人？', 'start': 10, 'end': 19}],
   'SHINT': [0, 1, 2, 4],
   'ANSWER': [{'ATEXT': '北宋',
     'ATOKEN': [{'text': '北宋', 'start': 40, 'end': 42}]}],
   'ASPAN': [{'text': '苏轼', 'start': 0, 'end': 2},
    {'text': '东坡居士', 'start': 72, 'end': 76},
    {'text': '北宋', 'start': 40, 'end': 42}],
   'keywords': []}],
 'DTEXT': '蘇軾（1037年1月8日－1101年8月24日），眉州眉山（今四川省眉山市）人，北宋時著名的文學家、政治家、藝術家、醫學家。字子瞻，一字和仲，號東坡居士、鐵冠道人。嘉佑二年進士，累官至端明殿學士兼翰林學士，禮部尚書。南宋理學方熾時，加賜諡號文忠，複追贈太師。有《東坡先生大全集》及《東坡樂府》詞集傳世，宋人王宗稷收其作品，編有《蘇文忠公全集》。\n其散文、詩、詞、賦均有成就，且善書法和繪畫，是文學藝術史上的通才，也是公認韻文散文造詣皆比較傑出的大家。蘇軾的散文為唐宋四家（韓愈、柳宗元、歐蘇）之末，與唐代的古文運動發起者韓愈並稱為「韓潮蘇海」，也與歐陽修並稱「歐蘇」；更與父親蘇洵、弟蘇轍合稱「三蘇」，父子三人，同列唐宋八大家。蘇軾之詩與黃庭堅並稱「蘇黃」，又與陸游並稱「蘇陸」；其詞「以詩入詞」，首開詞壇「豪放」一派，振作了晚唐、五代以來綺靡的西崑體餘風。後世與南宋辛棄疾

In [69]:
json_write(fgc_data, config.DATA_ROOT/'FGC_keyword_1.7.1'/'FGC_key_dev.json')